In [2]:
import pandas as pd
import geopandas as gpd
# from tabula import read_pdf

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# df_informacoes = read_pdf('data/censo_2010/BASE DE INFORMAåES POR SETOR CENSITµRIO Censo 2010 - Universo.pdf', stream=True, pages=46, lattice=True)

In [3]:
# df_informacoes[0]

In [4]:
## Malha de setores do BR
# https://www.ibge.gov.br/geociencias/downloads-geociencias.html?caminho=organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/2021/Malha_de_setores_(shp)_Brasil

In [5]:
## Download das tabelas e documentacao
# https://www.ibge.gov.br/estatisticas/sociais/populacao/9662-censo-demografico-2010.html?edicao=10410

# Selecao das variáveis

## Habitação einfraestrutura

* Porcentagem de domicílios do tipo cômodo
* Porcentagem de domicílios sem coleta de lixo
* Porcentagem de domicílios sem ligação à rede de abastecimento de água
* Porcentagem de domicílios sem banheiros ou sanitários
* Porcentagem de domicílios sem ligação à rede de esgoto ou fossa séptica
* Porcentagem de domicílios – outra forma de posse da moradia
* Número de banheiros por habitante

## Renda e escolaridade do responsável pelo domicílio

* Porcentagem de responsáveis por domicílio não alfabetizados
* Porcentagem de responsáveis por domicílio com menos de 30 anos não alfabetizados
* Porcentagem de responsáveis por domicílio com renda de até 3 salários mínimos
* Porcentagem de responsáveis por domicílio com menos de 8 anos de estudo
* Anos médios de estudo do responsável pelo domicílio
* Renda média do responsável pelo domicílio

## Aspectos demográficos

* Número de domicílios particulares permanentes no setor censitário
* Número de domicílios improvisados no setor censitário
* Número de pessoas residentes no setor censitário
* Porcentagem de responsáveis por domicílios com menos de 30 anos
* Número médio de pessoas por domicílio



In [6]:
# gdf_br = gpd.read_file('data/censo_2010/BR_Setores_2021.zip')

In [7]:
# gdf_br.shape

In [8]:
# gdf_br.NM_SIT.value_counts()

In [9]:
# gdf_br.columns

In [2]:
UFS = [
    'AC',
    'AL',
    'AM',
    'AP',
    'BA',
    'CE',
    'DF',
    'ES',
    'GO',
    'MA',
    'MG',
    'MS',
    'MT',
    'PA',
    'PB',
    'PE',
    'PI',
    'PR',
    'RJ',
    'RN',
    'RO',
    'RR',
    'RS',
    'SC',
    'SE',
    'SP1',
    'SP2',
    'TO'
]

In [4]:
## Hash para montar a base de dados
# {'file1': ['var1', 'var2', ..., 'var n'], ...,'file n': ['var1', 'var2', ..., 'var n']}

model_vars = {
    'Basico':{
        # 'Cod_setor': 'cod_setor',
        'Cod_Grandes Regiões': 'cod_grande_regiao',
        # 'Nome_Grande_Regiao': 'Nome GRande Região',
        'Cod_UF': 'uf',
        'Cod_meso': 'meso_regiao',
        'Cod_RM': 'rm',
        'Nome_da_RM': 'nome_rm',
        # 'Cod_Grandes Regiões': ''
        'Cod_municipio': 'cod_municipio',
        'Nome_do_municipio': 'nome_municipio',
        'Situacao_setor': 'situacao_setor',
        'Tipo_setor': 'tipo_do_setor',
        'V001': 'numero_de_domicilios',
        'V002': 'numero_moradores'
    },
    'Domicilio01': {
        'V003': 'domicilios_tipo_casa',
        'V002': 'domicilios_particulares_permanentes',
        'V035': 'domicilios_com_lixo_coletado',
        'V106': 'domicilios_tipo_casa_com_abastecimento_de_agua',
        'V023': 'domicilios_particulares_permanentes_sem_banheiro_ou_sanitario',
        'V017': 'domicilios_particulares_permanentes_com_banheiro_ligado_a_esgoto',
        'V018': 'domicilios_particulares_permanentes_com_banheiro_ligado_a_fossa_septica',
        'V025': 'domicilios_particulares_permanentes_com_1_banheiro',
        'V026': 'domicilios_particulares_permanentes_com_2_banheiros',
        'V027': 'domicilios_particulares_permanentes_com_3_banheiros',
        'V028': 'domicilios_particulares_permanentes_com_4_banheiros',
        'V029': 'domicilios_particulares_permanentes_com_5_banheiros',
        'V030': 'domicilios_particulares_permanentes_com_6_banheiros',
        'V031': 'domicilios_particulares_permanentes_com_7_banheiros',
        'V032': 'domicilios_particulares_permanentes_com_8_banheiros',
        'V033': 'domicilios_particulares_permanentes_com_mais_de_9_banheiros',
        'V010': 'domicilios_particulares_permanentes_cedidos_de_outra_forma',
        'V011': 'domicilios_particulares_permanentes_em_outra_condicao_de_ocupacao',
    },
    'Responsavel02': {
        'V001': 'pessoas_responsaveis',
        'V002': 'pessoas_responsaveis_com_10_anos',
        'V003': 'pessoas_responsaveis_com_11_anos',
        'V004': 'pessoas_responsaveis_com_12_anos',
        'V005': 'pessoas_responsaveis_com_13_anos',
        'V006': 'pessoas_responsaveis_com_14_anos',
        'V007': 'pessoas_responsaveis_com_15_anos',
        'V008': 'pessoas_responsaveis_com_16_anos',
        'V009': 'pessoas_responsaveis_com_17_anos',
        'V010': 'pessoas_responsaveis_com_18_anos',
        'V011': 'pessoas_responsaveis_com_19_anos',
        'V012': 'pessoas_responsaveis_com_20_anos',
        'V013': 'pessoas_responsaveis_com_21_anos',
        'V014': 'pessoas_responsaveis_com_22_anos',
        'V015': 'pessoas_responsaveis_com_23_anos',
        'V016': 'pessoas_responsaveis_com_24_anos',
        'V017': 'pessoas_responsaveis_com_25_anos',
        'V018': 'pessoas_responsaveis_com_26_anos',
        'V019': 'pessoas_responsaveis_com_27_anos',
        'V020': 'pessoas_responsaveis_com_28_anos',
        'V021': 'pessoas_responsaveis_com_29_anos',
        'V093': 'pessoas_responsaveis_alfabetizadas',
        'V094': 'pessoas_responsaveis_alfabetizadas_de_10_a_14_anos',
        'V095': 'pessoas_responsaveis_alfabetizadas_de_15_a_19_anos',
        'V096': 'pessoas_responsaveis_alfabetizadas_de_20_a_24_anos',
        'V097': 'pessoas_responsaveis_alfabetizadas_de_25_a_29_anos'
    },
    'DomicilioRenda':{
        'V001': 'total_domicilios_improvisados',
        'V002': 'total_rendimento_dos_domicilios',
    },
    'ResponsavelRenda':{
        'V001': 'responsaveis_com_renda_ate_meio_salario_minimo',
        'V002': 'responsaveis_com_renda_de_meio_a_1_salario_minimo',
        'V003': 'responsaveis_com_renda_de_1_a_2_salarios_minimos',
        'V004': 'responsaveis_com_renda_de_2_a_3_salarios_minimos'
    }
}


In [5]:
dfs = []
for uf in UFS:
    print(f'Processando {uf}')
    dfs_uf = []
    for file in model_vars:
        file_path = f'data/censo_2010/csvs/{file}_{uf}.csv'
        fields = list(model_vars[file].keys())
        fields.append('Cod_setor')
        try:
            df = pd.read_csv(file_path, delimiter=';', usecols=fields, dtype="string", encoding='UTF-8').set_index('Cod_setor')
        except:
            try:
                df = pd.read_csv(file_path, delimiter=';', usecols=fields, dtype="string", encoding='ISO-8859-1').set_index('Cod_setor')
            except:
                df = pd.read_csv(file_path, delimiter=',', usecols=fields, dtype="string", encoding='UTF-8').set_index('Cod_setor')
        df.rename(columns=model_vars[file], inplace=True)
        dfs_uf.append(df)
        # break
    df = pd.concat(dfs_uf, axis=1)
    df.loc[:, 'nome_UF'] = uf
    dfs.append(df)

Processando AC
Processando AL
Processando AM
Processando AP
Processando BA
Processando CE
Processando DF
Processando ES
Processando GO
Processando MA
Processando MG
Processando MS
Processando MT
Processando PA
Processando PB
Processando PE
Processando PI
Processando PR
Processando RJ
Processando RN
Processando RO
Processando RR
Processando RS
Processando SC
Processando SE
Processando SP1
Processando SP2
Processando TO


In [6]:
df = pd.concat(dfs)

In [7]:
df

,cod_grande_regiao,uf,meso_regiao,rm,nome_rm,cod_municipio,nome_municipio,situacao_setor,tipo_do_setor,numero_de_domicilios,...,pessoas_responsaveis_alfabetizadas_de_15_a_19_anos,pessoas_responsaveis_alfabetizadas_de_20_a_24_anos,pessoas_responsaveis_alfabetizadas_de_25_a_29_anos,total_domicilios_improvisados,total_rendimento_dos_domicilios,responsaveis_com_renda_ate_meio_salario_minimo,responsaveis_com_renda_de_meio_a_1_salario_minimo,responsaveis_com_renda_de_1_a_2_salarios_minimos,responsaveis_com_renda_de_2_a_3_salarios_minimos,nome_UF
Cod_setor,,,,,,,,,,,,,,,,,,,,,
120001305000001,1,12,1202,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,1,0,282,...,2,19,23,0,601805,17,89,63,25,AC
120001305000002,1,12,1202,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,1,0,342,...,9,24,50,2,385433,30,139,102,19,AC
120001305000003,1,12,1202,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,1,0,482,...,18,47,72,1,532208,42,193,98,28,AC
120001305000004,1,12,1202,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,5,0,53,...,1,3,7,0,58853,3,14,21,8,AC
120001305000005,1,12,1202,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,8,0,96,...,0,3,13,0,112979,6,47,25,4,AC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170210905000098,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,4,33,63,0,484164,13,100,162,35,TO
170210905000099,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,6,23,56,1,341945,11,165,85,16,TO
170210905000100,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,4,21,52,0,441825,10,178,77,25,TO


In [8]:
df_areas = pd.read_csv('resultados/area_dos_setores.zip', dtype={'CD_GEOCODI':'object'})

In [9]:
df_areas.set_index('CD_GEOCODI', inplace=True)

In [10]:
df = df.join(df_areas, how='left')

In [11]:
df['area_do_setor'] = df['area_do_setor'].fillna(1.)

In [12]:
df.shape

(310120, 63)

In [13]:
df.nome_rm.value_counts()

Municípios não pertencentes a estrutura de RM        173563
RM São Paulo                                          29943
RM Rio de Janeiro                                     19507
RM Belo Horizonte                                      8523
RM Porto Alegre                                        6971
RIDE - Região Integrada de Desenvolvimento do ...      6059
RM Salvador                                            5023
RM Recife                                              4535
RM Fortaleza                                           4480
RM Curitiba                                            4348
RM Campinas                                            4113
RM Baixada Santista                                    2968
RM Manaus                                              2899
RM Goiânia                                             2882
RM Grande Vitória                                      2838
Munic¡pios não pertencentes a estrutura de RM          2089
RM Belém                                

In [14]:
len(df.rm.value_counts().to_list())

41

In [15]:
df.dtypes

cod_grande_regiao                                     string
uf                                                    string
meso_regiao                                           string
rm                                                    string
nome_rm                                               string
                                                      ...   
responsaveis_com_renda_de_meio_a_1_salario_minimo     string
responsaveis_com_renda_de_1_a_2_salarios_minimos      string
responsaveis_com_renda_de_2_a_3_salarios_minimos      string
nome_UF                                               object
area_do_setor                                        float64
Length: 63, dtype: object

In [16]:
CAMPOS_MODELO = [
    'cod_grande_regiao',
    'uf',
    'nome_UF',
    'rm',
    'nome_rm',
    'meso_regiao',
    'cod_municipio',
    'nome_municipio',
    'situacao_setor',
    'tipo_do_setor',
    'numero_de_domicilios',
    'numero_moradores',
    'area_do_setor',
    'porcentagem_domicilios_sem_coleta_de_lixo',
    'porcentagem_domicilios_sem_ligacao_rede_de_agua',
    'porcentagem_domicilios_sem_banheiros',
    'porcentagem_domicilios_sem_esgoto_ou_fossa',
    # # 'porcentagem_domicilios_tipo_comodo',
    'porcentagem_domicilios_cedidos_outra_forma',
    'porcentagem_domicilios_em_outra_condicao_de_ocupacao',
    'numero_de_banheiros_por_habitante',
    'porcentagem_responsaveis_nao_alfabetizado',
    'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos',
    'porcentagem_responsaveis_renda_ate_3_salarios',
    # # 'porcentagem_responsaveis_menos_8_anos_de_estudo',
    # # 'anos_medios_estudo_responsavel',
    'renda_media_responsavel',
    'numero_de_domicilios_particulares_permanentes',
    'numero_de_domicilios_improvisados',
    'numero_de_pessoas_residentes',
    'porcentagem_de_responsaveis_com_menos_30_anos',
    'numero_medio_pessoas', 
    'densidade_populacional',
    'densidade_domicilios'
]

In [17]:
df.loc[:, 'porcentagem_domicilios_sem_coleta_de_lixo'] = pd.to_numeric(df.loc[:, 'domicilios_com_lixo_coletado'], errors='coerce') / pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes'], errors='coerce')

In [18]:
df.loc[:, 'porcentagem_domicilios_sem_ligacao_rede_de_agua'] = (pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes'], errors='coerce') - pd.to_numeric(df.loc[:, 'domicilios_tipo_casa_com_abastecimento_de_agua'], errors='coerce')) / pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes'], errors='coerce')

In [19]:
df.loc[:, 'porcentagem_domicilios_sem_banheiros'] = pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes_sem_banheiro_ou_sanitario'], errors='coerce') / pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes'], errors='coerce')

In [20]:
# 'porcentagem_domicilios_sem_esgoto_ou_fossa'
df.loc[:, 'porcentagem_domicilios_sem_esgoto_ou_fossa'] = (pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes'], errors='coerce') - pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes_com_banheiro_ligado_a_esgoto'], errors='coerce') - pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes_com_banheiro_ligado_a_fossa_septica'], errors='coerce')) / pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes'], errors='coerce')


In [21]:
# porcentagem_domicilios_cedidos_outra_forma
df.loc[:, 'porcentagem_domicilios_cedidos_outra_forma'] = pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes_cedidos_de_outra_forma'], errors='coerce') / pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes'], errors='coerce')

In [22]:
# porcentagem_domicilios_em_outra_condicao_de_ocupacao
df.loc[:, 'porcentagem_domicilios_em_outra_condicao_de_ocupacao'] = pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes_em_outra_condicao_de_ocupacao'], errors='coerce') / pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes'], errors='coerce')

In [23]:
banheiros = [
    'domicilios_particulares_permanentes_com_1_banheiro',
    'domicilios_particulares_permanentes_com_2_banheiros',
    'domicilios_particulares_permanentes_com_3_banheiros',
    'domicilios_particulares_permanentes_com_4_banheiros',
    'domicilios_particulares_permanentes_com_5_banheiros',
    'domicilios_particulares_permanentes_com_6_banheiros',
    'domicilios_particulares_permanentes_com_7_banheiros',
    'domicilios_particulares_permanentes_com_8_banheiros',
    'domicilios_particulares_permanentes_com_mais_de_9_banheiros'
]

df.loc[:, 'banheiros'] = 0

for k,v in list(zip(range(9), banheiros)):
    df.loc[:, 'banheiros'] += pd.to_numeric(df.loc[:, v], errors='coerce') * (k+1)

In [24]:
# numero_de_banheiros_por_habitante
df.loc[:, 'numero_de_banheiros_por_habitante'] = df.banheiros / pd.to_numeric(df.numero_moradores, errors='coerce')

In [25]:
# porcentagem_responsaveis_nao_alfabetizado
responsaveis_nao_alfabetizados = (pd.to_numeric(df.loc[:, 'pessoas_responsaveis'], errors='coerce') - pd.to_numeric(df.loc[:, 'pessoas_responsaveis_alfabetizadas'], errors='coerce'))
df.loc[:, 'porcentagem_responsaveis_nao_alfabetizado'] =  responsaveis_nao_alfabetizados / pd.to_numeric(df.loc[:, 'pessoas_responsaveis'], errors='coerce')

In [26]:
# porcentagem_de_responsaveis_com_menos_30_anos
        
responsaveis_menos_30 = [
        'pessoas_responsaveis_com_10_anos',
        'pessoas_responsaveis_com_11_anos',
        'pessoas_responsaveis_com_12_anos',
        'pessoas_responsaveis_com_13_anos',
        'pessoas_responsaveis_com_14_anos',
        'pessoas_responsaveis_com_15_anos',
        'pessoas_responsaveis_com_16_anos',
        'pessoas_responsaveis_com_17_anos',
        'pessoas_responsaveis_com_18_anos',
        'pessoas_responsaveis_com_19_anos',
        'pessoas_responsaveis_com_20_anos',
        'pessoas_responsaveis_com_21_anos',
        'pessoas_responsaveis_com_22_anos',
        'pessoas_responsaveis_com_23_anos',
        'pessoas_responsaveis_com_24_anos',
        'pessoas_responsaveis_com_25_anos',
        'pessoas_responsaveis_com_26_anos',
        'pessoas_responsaveis_com_27_anos',
        'pessoas_responsaveis_com_28_anos',
        'pessoas_responsaveis_com_29_anos',
]

df.loc[:, 'responsaveis_menos_30'] = 0

for v in responsaveis_menos_30:
    df.loc[:, 'responsaveis_menos_30'] += pd.to_numeric(df.loc[:, v], errors='coerce')

df.loc[:, 'porcentagem_de_responsaveis_com_menos_30_anos'] = df.responsaveis_menos_30 / pd.to_numeric(df.loc[:, 'pessoas_responsaveis'], errors='coerce')

In [27]:
# porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos
alfabetizados_menos_30 = [
    'pessoas_responsaveis_alfabetizadas_de_10_a_14_anos',
    'pessoas_responsaveis_alfabetizadas_de_15_a_19_anos',
    'pessoas_responsaveis_alfabetizadas_de_20_a_24_anos',
    'pessoas_responsaveis_alfabetizadas_de_25_a_29_anos'
]

df.loc[:, 'alfabetizados_menos_30'] = 0

for v in alfabetizados_menos_30:
    df.loc[:, 'alfabetizados_menos_30'] += pd.to_numeric(df.loc[:, v], errors='coerce')

df.loc[:, 'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos'] = (df.loc[:, 'responsaveis_menos_30'] - df.loc[:, 'alfabetizados_menos_30']) / df.loc[:, 'responsaveis_menos_30']

In [28]:
# porcentagem_responsaveis_renda_ate_3_salarios
responsaveis_renda_ate_3_salarios = [
        'responsaveis_com_renda_ate_meio_salario_minimo',
        'responsaveis_com_renda_de_meio_a_1_salario_minimo',
        'responsaveis_com_renda_de_1_a_2_salarios_minimos',
        'responsaveis_com_renda_de_2_a_3_salarios_minimos'
]

df.loc[:, 'responsaveis_renda_ate_3_salarios'] = 0

for v in responsaveis_renda_ate_3_salarios:
    df.loc[:, 'responsaveis_renda_ate_3_salarios'] += pd.to_numeric(df.loc[:, v], errors='coerce')

df.loc[:, 'porcentagem_responsaveis_renda_ate_3_salarios'] = (df.loc[:, 'responsaveis_renda_ate_3_salarios']) / pd.to_numeric(df.loc[:, 'pessoas_responsaveis'], errors='coerce')

In [29]:
# renda_media_responsavel
df.loc[:, ['total_rendimento_dos_domicilios', 'numero_de_domicilios']]
renda_por_domicilio = pd.to_numeric(df.loc[:, 'total_rendimento_dos_domicilios'], errors='coerce') / pd.to_numeric(df.loc[:, 'numero_de_domicilios'], errors='coerce')
df.loc[:, 'renda_media_responsavel'] = pd.to_numeric(df.loc[:, 'total_rendimento_dos_domicilios'], errors='coerce') / pd.to_numeric(df.loc[:, 'pessoas_responsaveis'], errors='coerce')


In [30]:
# numero_de_domicilios_particulares_permanentes
# domicilios_particulares_permanentes
df.loc[:, 'numero_de_domicilios_particulares_permanentes'] = pd.to_numeric(df.loc[:, 'domicilios_particulares_permanentes'], errors='coerce')

In [31]:
# numero_de_domicilios_improvisados
# total_domicilios_improvisados
df.loc[:, 'numero_de_domicilios_improvisados'] = pd.to_numeric(df.loc[:, 'total_domicilios_improvisados'], errors='coerce')

In [32]:
# numero_de_pessoas_residentes
# numero_moradores
df.loc[:, 'numero_de_pessoas_residentes'] = pd.to_numeric(df.loc[:, 'numero_moradores'], errors='coerce')

In [33]:
# numero_medio_pessoas'
df.loc[:, 'numero_medio_pessoas'] = pd.to_numeric(df.loc[:, 'numero_moradores'], errors='coerce') / pd.to_numeric(df.loc[:, 'numero_de_domicilios'], errors='coerce')

In [34]:
df.loc[:, 'densidade_populacional'] = pd.to_numeric(df.loc[:, 'numero_moradores'], errors='coerce') / df.loc[:, 'area_do_setor']

In [35]:
df.loc[:, 'densidade_domicilios'] = pd.to_numeric(df.loc[:, 'numero_de_domicilios'], errors='coerce') / df.loc[:, 'area_do_setor']

In [36]:
df

,cod_grande_regiao,uf,meso_regiao,rm,nome_rm,cod_municipio,nome_municipio,situacao_setor,tipo_do_setor,numero_de_domicilios,...,porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos,responsaveis_renda_ate_3_salarios,porcentagem_responsaveis_renda_ate_3_salarios,renda_media_responsavel,numero_de_domicilios_particulares_permanentes,numero_de_domicilios_improvisados,numero_de_pessoas_residentes,numero_medio_pessoas,densidade_populacional,densidade_domicilios
Cod_setor,,,,,,,,,,,,,,,,,,,,,
120001305000001,1,12,1202,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,1,0,282,...,0.022222,194.0,0.687943,2134.060284,282,0.0,957.0,3.393617,957.0,282.0
120001305000002,1,12,1202,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,1,0,342,...,0.011905,290.0,0.843023,1120.444767,342,2.0,1203.0,3.517544,1203.0,342.0
120001305000003,1,12,1202,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,1,0,482,...,0.021127,361.0,0.747412,1101.879917,482,1.0,1700.0,3.526971,1700.0,482.0
120001305000004,1,12,1202,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,5,0,53,...,0.083333,46.0,0.867925,1110.433962,53,0.0,182.0,3.433962,182.0,53.0
120001305000005,1,12,1202,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,8,0,96,...,0.157895,82.0,0.854167,1176.864583,96,0.0,305.0,3.177083,305.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170210905000098,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.000000,310.0,0.863510,1348.646240,359,0.0,NaN,NaN,NaN,NaN
170210905000099,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.000000,277.0,0.780282,963.225352,354,1.0,NaN,NaN,NaN,NaN
170210905000100,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.012821,290.0,0.857988,1307.174556,338,0.0,NaN,NaN,NaN,NaN


In [37]:
df.loc[:, CAMPOS_MODELO].to_csv('resultados/modelo.csv.zip', compression='zip')

In [3]:
df_ = pd.read_csv('resultados/modelo.csv.zip', compression='zip')

/tmp/ipykernel_120522/3604218728.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ = pd.read_csv('resultados/modelo.csv.zip', compression='zip')


In [4]:
df_.columns

Index(['Cod_setor', 'cod_grande_regiao', 'uf', 'nome_UF', 'rm', 'nome_rm',
       'meso_regiao', 'cod_municipio', 'nome_municipio', 'situacao_setor',
       'tipo_do_setor', 'numero_de_domicilios', 'numero_moradores',
       'area_do_setor', 'porcentagem_domicilios_sem_coleta_de_lixo',
       'porcentagem_domicilios_sem_ligacao_rede_de_agua',
       'porcentagem_domicilios_sem_banheiros',
       'porcentagem_domicilios_sem_esgoto_ou_fossa',
       'porcentagem_domicilios_cedidos_outra_forma',
       'porcentagem_domicilios_em_outra_condicao_de_ocupacao',
       'numero_de_banheiros_por_habitante',
       'porcentagem_responsaveis_nao_alfabetizado',
       'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos',
       'porcentagem_responsaveis_renda_ate_3_salarios',
       'renda_media_responsavel',
       'numero_de_domicilios_particulares_permanentes',
       'numero_de_domicilios_improvisados', 'numero_de_pessoas_residentes',
       'porcentagem_de_responsaveis_com_menos_30